# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2024-09-06	Finding Complete Impossible Differential Attacks on AndRX Ciphers and Efficient Distinguishers for ARX Designs	IACR Transactions on Symmetric Cryptology	This paper explores advanced techniques for impossible differential attacks on AndRX ciphers and efficient distinguishers for ARX designs.	Debasmita Chakraborty, Hosein Hadipour, Phuong Hoa Nguyen, Maria Eichlseder. (2024). "Finding Complete Impossible Differential Attacks on AndRX Ciphers and Efficient Distinguishers for ARX Designs." <i>IACR Transactions on Symmetric Cryptology, TOSC-2024(3)</i>.	finding-impossible-differential-andrx	https://doi.org/10.46586/tosc.v2024.i3.84-176	
2023-01-25	Revisiting Related-Key Boomerang Attacks on AES Using Computer-Aided Tool	Advances in Cryptology – ASIACRYPT 2022	This paper revisits related-key boomerang attacks on AES with a focus on utilizing computer-aided tools for enhanced cryptanalysis.	Patrick Derbez, Marie Eu

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2024-09-06,Finding Complete Impossible Differential Attac...,IACR Transactions on Symmetric Cryptology,This paper explores advanced techniques for im...,"Debasmita Chakraborty, Hosein Hadipour, Phuong...",finding-impossible-differential-andrx,https://doi.org/10.46586/tosc.v2024.i3.84-176,NaN
1,2023-01-25,Revisiting Related-Key Boomerang Attacks on AE...,Advances in Cryptology – ASIACRYPT 2022,This paper revisits related-key boomerang atta...,"Patrick Derbez, Marie Euler, Pierre-Alain Fouq...",revisiting-related-key-boomerang-aes,https://doi.org/10.1007/978-3-031-22969-5_3,NaN
2,2020-04-11,Solving the Clustered Traveling Salesman Probl...,International Transactions in Operational Rese...,This paper addresses the clustered traveling s...,"Minh Hoàng Hà, Hoa Nguyen Phuong, Huyen Tran N...",solving-clustered-traveling-salesman,https://doi.org/10.1111/itor.12797,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [9]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [8]:
!ls ../_publications/

2020-04-11-solving-clustered-traveling-salesman.md
2023-01-25-revisiting-related-key-boomerang-aes.md
2024-09-06-finding-impossible-differential-andrx.md


In [7]:
!cat ../_publications/2024-09-06-finding-impossible-differential-andrx.md

---
title: "Finding Complete Impossible Differential Attacks on AndRX Ciphers and Efficient Distinguishers for ARX Designs"
collection: publications
permalink: /publication/2024-09-06-finding-impossible-differential-andrx
excerpt: 'This paper explores advanced techniques for impossible differential attacks on AndRX ciphers and efficient distinguishers for ARX designs.'
date: 2024-09-06
venue: 'IACR Transactions on Symmetric Cryptology'
paperurl: 'https://doi.org/10.46586/tosc.v2024.i3.84-176'
citation: 'Debasmita Chakraborty, Hosein Hadipour, Phuong Hoa Nguyen, Maria Eichlseder. (2024). &quot;Finding Complete Impossible Differential Attacks on AndRX Ciphers and Efficient Distinguishers for ARX Designs.&quot; <i>IACR Transactions on Symmetric Cryptology, TOSC-2024(3)</i>.'
---
This paper explores advanced techniques for impossible differential attacks on AndRX ciphers and efficient distinguishers for ARX designs.

[Download paper here](https://doi.org/10.46586/tosc.v2024.i3.84-176)

Rec